In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.metrics import r2_score

In [2]:
categorical=['PULocationID','DOLocationID']

def read_dataframe(filename):
    df=pd.read_parquet(filename)
    df['duration']=df.tpep_dropoff_datetime -df.tpep_pickup_datetime
    df.duration=df.duration.apply(lambda td: td.total_seconds()/60)
    df=df[(df.duration>=1) & (df.duration<=60)]
    df[categorical]=df[categorical].astype(str)
    
    return df

## Q1. Downloading the data
#### We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

#### Download the data for January and February 2023.

#### Read the data for January. How many columns are there?

In [3]:
hw_df1= pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [4]:
col_names =hw_df1.columns

In [5]:
len(col_names)

19

In [6]:
hw_df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q2. Computing duration
#### Now let's compute the duration variable. It should contain the duration of a ride in minutes.

#### What's the standard deviation of the trips duration in January?

In [7]:
hw_df1['duration']=hw_df1.tpep_dropoff_datetime -hw_df1.tpep_pickup_datetime
hw_df1.duration=hw_df1.duration.apply(lambda td: td.total_seconds()/60)

In [8]:
hw_df1['duration'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [9]:
round(hw_df1.duration.describe(),2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

##### Standard Deviation is  42.59 mins

## Q3. Dropping outliers
#### Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them #### and keep only the records where the duration was between 1 and 60 minutes (inclusive).

#### What fraction of the records left after you dropped the outliers?

In [10]:
hw_df2=hw_df1[(hw_df1.duration>=1) & (hw_df1.duration<=60)]

In [11]:
hw_df2.duration.count()

3009173

In [12]:
hw_df1.duration.count()

3066766

In [13]:
round(3009173/3066766 *100,0)

98.0

##### 98% of records are left

## Q4. One-hot encoding
#### Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

#### Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
#### Fit a dictionary vectorizer
#### Get a feature matrix from it
#### What's the dimensionality of this matrix (number of columns)?

In [14]:
catagorical=['PULocationID','DOLocationID']

In [15]:
hw_df2.isnull().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71105
trip_distance                0
RatecodeID               71105
store_and_fwd_flag       71105
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71105
airport_fee              71105
duration                     0
dtype: int64

In [16]:
hw_df2[catagorical]=hw_df2[catagorical].astype('str')

C:\Users\kitra\AppData\Local\Temp\ipykernel_11352\1733657103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hw_df2[catagorical]=hw_df2[catagorical].astype('str')


In [22]:
hw_dicts=hw_df2[catagorical].to_dict(orient='records')

In [23]:
hw_dicts[:10]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [24]:
dv=DictVectorizer()

In [25]:
hw_df_transform= dv.fit_transform(hw_dicts)

In [26]:
hw_df_transform.shape

(3009173, 515)

In [27]:
len(dv.feature_names_)

515

##### 515 is the number of columns

## Q5. Training a model
#### Now let's use the feature matrix from the previous step to train a model.

#### Train a plain linear regression model with default parameters
#### Calculate the RMSE of the model on the training data
#### What's the RMSE on train?

In [29]:
X_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')

In [30]:
y_train=X_train.duration.values

In [31]:
train_dicts=X_train[catagorical].to_dict(orient='records')

In [32]:
X_train=dv.fit_transform(train_dicts)

In [33]:
X_train.shape

(3009173, 515)

In [34]:
lr= LinearRegression()

In [35]:
lr.fit(X_train,y_train)

LinearRegression()

In [36]:
y_pred=lr.predict(X_train)

In [47]:
round(mean_squared_error(y_train,y_pred,squared=False),4)

7.6493

#### RMSE on Train 7.6493

## Q6. Evaluating the model
#### Now let's apply this model to the validation dataset (February 2023).

#### What's the RMSE on validation?

In [38]:
X_val= read_dataframe('data/yellow_tripdata_2023-02.parquet')

In [39]:
y_val=X_val.duration.values

In [40]:
val_dicts=X_val[catagorical].to_dict(orient='records')

In [41]:
X_val=dv.fit_transform(val_dicts)

In [42]:
X_val.shape

(2855951, 514)

In [43]:
vlr= LinearRegression()

In [44]:
vlr.fit(X_val,y_val)

LinearRegression()

In [45]:
y_val_pred=vlr.predict(X_val)

In [48]:
round(mean_squared_error(y_val,y_val_pred,squared=False),4)

7.7789

#### RMSE for validation dataset is 7.7789